This is the inference using the model created in the following notebook.
https://www.kaggle.com/sreejaej/attention-is-all-you-need-score-of-0-532





In [ ]:
import pandas as pd
import numpy as np
import json
import re
import torch.nn as nn 
import torch
from transformers import BertTokenizer
from nltk.corpus import stopwords
import os

In [ ]:
MAX_SEQ=500

In [ ]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
train = pd.read_csv(train_path)
train


In [ ]:
train['cleaned_label'] = train['cleaned_label'].apply(lambda x : x.strip())
known_labels = np.unique ( list( train['cleaned_label'].values ))

In [ ]:
'''known_labels_final=[]
for label1 in known_labels:
    ignore=False
    for label2 in known_labels:
        
        if (label1==label2):
            continue
        elif ( label1 in label2):
            ignore=True
            print( "ignoring", label1,"coz of", label2 )
            break
    if ( ignore == False ) :
        known_labels_final.append(label1)

known_labels_final'''
     

In [ ]:
test_path = '../input/coleridgeinitiative-show-us-the-data/sample_submission.csv'
paper_test_folder = '../input/coleridgeinitiative-show-us-the-data/test'
test = pd.read_csv(test_path)
test=test.set_index(['Id'])
test

In [ ]:
from tqdm import tqdm

stopwords_set = set(stopwords.words('english')) 
def clean_text(input_words):
    #return [   re.sub('[^A-Za-z0-9]+', ' ', str(r).lower()).strip() for r in input_words if not r.lower() in stopwords_set  ]
    return [   re.sub('[^A-Za-z0-9\[\]]+', ' ', str(r).lower()).strip() for r in input_words  ]



def process_row_with_bert(words, tokenizer, stopwords_set):  
        
    encoded_input_word_list = clean_text(words)
    encoded_input_word_list = list(map(tokenizer.convert_tokens_to_ids, tokenizer.tokenize ( " ".join(encoded_input_word_list)) ))
    

    if ( len(encoded_input_word_list) < MAX_SEQ ):
        encoded_input_word_list = encoded_input_word_list + list(np.zeros( MAX_SEQ - len(encoded_input_word_list)) )
    else :
        encoded_input_word_list = encoded_input_word_list[0:MAX_SEQ]
    
     
    return encoded_input_word_list

def process_data_with_bert (df, tokenizer) :
    max_word_count = 0 
    text_array = [] 
    code_array=[]
     
    stopwords_set = set(stopwords.words('english')) 
    
    df_final = pd.DataFrame(columns = ['Id', 'text'])
    
    for index, id in tqdm ( enumerate ( df.index) )  :
        with open(paper_test_folder+'/'+ id +'.json', 'r') as f:
                #Load the json 
                paper = json.load(f)
                
                #Convert the relevant sentences into a single paragraph
                section_sentences =[]
                for section_index in range ( 0, len(paper)):
                    section_sentences = section_sentences +  re.split('[.;\n•]',paper[section_index].get('text'))   
                section_sentences = " [sep] ".join(section_sentences) 
                sentence_words = section_sentences.split() + ['[sep]']

                # Chunk the paragraph into size of 500. Todo:Relook at this 
                n = int(0.7*MAX_SEQ) 
                word_chunks = [sentence_words[i:i+n+20] for i in range(0, len(sentence_words), n)] 
                word_chunks[-1] = word_chunks[-1] + list(np.empty( MAX_SEQ - len(word_chunks[-1]), dtype=str) )
                     
                # Add the words in the labels to the output wordlist
                #labels = df.loc[id]

                # Encode/Embed the words for processing
                df_new = pd.DataFrame(columns = ['Id', 'text', 'known_labels'])
                for i in range ( 0, len(word_chunks)):
                    encoded_input_word_list = process_row_with_bert(word_chunks[i], tokenizer, stopwords_set )
                    word_chunk_sentence = " ".join( clean_text( word_chunks[i] ) ) 
                    labels = [known_label for known_label in known_labels  if " ".join(clean_text( known_label.split() )) in word_chunk_sentence.lower() ] 
                    df_temp = pd.DataFrame ({"Id":id, "org_text": word_chunk_sentence, "text":[encoded_input_word_list], "known_labels": "|".join(labels) }, index=[i])
                    df_new =df_new.append(df_temp, ignore_index=True)
                    #print(df_new)

        df_final = df_final.append(df_new, ignore_index=True)

    return df_final

In [ ]:
import random 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained("../input/pretrained-bert-including-scripts/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12/")
tokenizer.add_tokens('[cls]')
tokenizer.add_tokens('[sep]')

df_test = process_data_with_bert(test, tokenizer)  
   

In [ ]:
df_test

In [ ]:
df_test[ df_test['known_labels'] != ""]

In [ ]:
import torch.nn as nn 

class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)

class ColleridgeTransformerModel(nn.Module):

    def __init__(self, embed_dim = 200, max_seq=MAX_SEQ, input_vocab_len=30523 , output_vocab_len= 30523): #HDKIM 100->MAX_SEQ
        super(ColleridgeTransformerModel, self).__init__()
        self.embed_dim = embed_dim
        self.embedding = nn.Embedding(input_vocab_len+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq, embed_dim)
        self.e_embedding = nn.Embedding(output_vocab_len+1, embed_dim)
        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 
        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, output_vocab_len+1)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, generated_words):
        device = x.device       
        
        # Embed the question
        x = self.embedding(x)
        
        # Calculate the position id and embed it
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        
        #Add it to the poisition id
        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x
        
        # This is the target_id/query ( right shifted input )
        e = self.e_embedding(generated_words) 
        pos_e_id = torch.arange(e.size(1)).unsqueeze(0).to(device)
        pos_e = self.pos_embedding(pos_e_id)
        e = e + pos_e
         
        # Send the query key and value to the multi attention network 
        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        att_output, att_weight = self.multi_att(x, x, x)
        x = self.layer_normal(att_output + x)
        
        
        for i in range ( 0, 1):
            
            e = e.permute(1, 0, 2)
              
            att_mask = future_mask(e.size(0)).to(device) 

            # Send the query key and value to the multi attention network 
            att_output, att_weight = self.multi_att(e, e, e, attn_mask=att_mask)

            e = self.layer_normal(att_output + e)

            # Send the query key and value to the multi attention network 
            att_output, att_weight = self.multi_att(e, x, x)

            # Normalization 
            att_output = self.layer_normal(att_output + e)

            # Reshape the output
            att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

            # Send the output to FFN.
            e = self.ffn(att_output)

            # 
            e = self.layer_normal(e + att_output)
        
        # 
        e = self.pred(e)
        e = self.softmax(e)
        e = e.permute(0, 2, 1)
        return e, att_weight

In [ ]:
xfrmer_model = ColleridgeTransformerModel(  )
xfrmer_model.load_state_dict(torch.load("../input/coleridgemodel/xform-463-fold0.model", map_location=torch.device(device) ))
xfrmer_model.eval()

In [ ]:
from torch.utils.data import Dataset, DataLoader


class ColleridgeDatasetEval(Dataset):
    def __init__(self, df, max_seq=MAX_SEQ):
        super(ColleridgeDatasetEval, self).__init__()
        self.max_seq = max_seq
        self.samples = df

    def __len__(self):
        return len(self.samples.index)
    
    def __getitem__(self, index):
        x = torch.tensor(self.samples.iloc[index]['text'], dtype=torch.int, device=device).view(-1, 1)
        y = torch.tensor(np.zeros(MAX_SEQ), dtype=torch.int, device=device).view(-1, 1)
        y[0] = 30522
        target_id = y[:-1]
        label = self.samples.iloc[index]['known_labels']
        
        return  x.squeeze(-1), target_id.squeeze(-1), label, self.samples.iloc[index]['org_text']

In [ ]:
dataset = ColleridgeDatasetEval(df_test)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

num=-1
xfrmer_model.to(device)

df_eval = pd.DataFrame(columns = ['id', 'predicted', 'known_labels'])
for item in dataloader:
    stop = False 
    i = 0
    num = num+1
    while ( i < 498 ):
        output,_ = xfrmer_model( item[0].to(device).long(),item[1].to(device).long()  )
        y = torch.argmax( output , dim=1)
        z = y[0][i]
        item[1][0][i+1]=z
        i = i+1
        if ( z == 0.0):
            break
    y = [30522] + y.tolist()[0]
    
    predicted_text = tokenizer.convert_tokens_to_string (tokenizer.convert_ids_to_tokens( [int(z) for z in y if z not in [30522, 0.0, 0]] ))
    complete_text= item[3][0]

    for index, label in enumerate ( predicted_text.split('[sep]') )   :  
            if(label in complete_text and label !=""):
                    #workaround-tobe fixed later
                    '''if( label not in item[2] ):
                        print(label)'''
                    
                    df_temp = pd.DataFrame ({ "id":df_test.iloc[num]['Id'],  "predicted": [label], "known_labels": [label]  }, index=[i])
                    df_eval =df_eval.append(df_temp, ignore_index=True)
            else:
                    #workaround-tobe fixed later
                    df_temp = pd.DataFrame ({ "id":df_test.iloc[num]['Id'],  "predicted": [""] , "known_labels": [""] }, index=[i])
                    df_eval =df_eval.append(df_temp, ignore_index=True)
            
    
    '''if(predicted_text=="[sep]"):
        for known_label1 in item[2][0].split("|"):
            ignore=False
            for known_label2 in item[2][0].split("|"):
                if( known_label1 in known_label2 and known_label1 != known_label2):
                    #print(known_label1, known_label2)
                    ignore=True
                    continue

            if (ignore==False):
                    df_temp = pd.DataFrame ({ "id":df_test.iloc[num]['Id'],  "predicted": [known_label1], "known_labels": [known_label1]  }, index=[i])
                    df_eval =df_eval.append(df_temp, ignore_index=True)
    else:
    
        for index, label in enumerate ( predicted_text.split('[sep]') )   :  

            if(label in complete_text and label !=""):
                    #workaround-tobe fixed later
                    df_temp = pd.DataFrame ({ "id":df_test.iloc[num]['Id'],  "predicted": [label], "known_labels": [label]  }, index=[i])
                    df_eval =df_eval.append(df_temp, ignore_index=True)

            elif label != "":
                for known_label1 in item[2][0].split("|"):
                    ignore=False
                    for known_label2 in item[2][0].split("|"):
                        if( known_label1 in known_label2 and known_label1 != known_label2):
                            #print(known_label1, known_label2)
                            ignore=True
                            continue

                    if (ignore==False):
                            df_temp = pd.DataFrame ({ "id":df_test.iloc[num]['Id'],  "predicted": [known_label1], "known_labels": [known_label1]  }, index=[i])
                            df_eval =df_eval.append(df_temp, ignore_index=True) 
            '''

df_eval

In [ ]:
df_eval[df_eval['known_labels']!=""]

In [ ]:
df_temp1=df_eval.groupby('id').apply(lambda row:  "|".join( np.unique( row['predicted'].values ) ) ) 
df_temp2=df_eval.groupby('id').apply(lambda row:  "|".join(np.unique(("|".join(row['known_labels'].values)).split("|")))  ) 


df_final_result = pd.DataFrame(columns = ['Id', 'PredictionString'])
df_final_result['Id'] = df_temp1.index
df_final_result['PredictionString1'] = list(map(lambda i: "|".join( clean_text(df_temp1[i].split("|")) ) , range(0,len(df_temp1))))
df_final_result['PredictionString2'] = list(map(lambda i: "|".join( clean_text(df_temp2[i].split("|")) ) , range(0,len(df_temp2))))

In [ ]:
#df_final_result['PredictionString'] = list(map( lambda x: "|".join ( np.unique(x.split("|"))[1:] ) , (df_final_result['PredictionString1']+"|"+df_final_result['PredictionString2']) ))
df_final_result['PredictionString'] = list(map( lambda x: "|".join ( np.unique(x.split("|")) ) , (df_final_result['PredictionString1'] )))

In [ ]:
#df_final_result['PredictionString'] =df_final_result['PredictionString'] + "|" + df_final_result['PredictionString'] 

In [ ]:
df_final_result['PredictionString'] .iloc[0]

In [ ]:
df_final_result['PredictionString'] =df_final_result['PredictionString'].apply(  lambda x: "|".join( string for string in x.split("|")\
                                                                                    if string != "") )



In [ ]:
df_final_result['PredictionString'] =df_final_result['PredictionString']+"|"+df_final_result['PredictionString']

In [ ]:
df_final_result[['Id','PredictionString']]

In [ ]:
df_final_result[['Id','PredictionString']].to_csv('/kaggle/working/submission.csv', index=False)